In [282]:
from functools import reduce

class Flower:
                
    def get_index(self, l):
        return reduce(lambda a, b: a * 3 + b, l, 0)
    
    def __init__(self, name, n, m):
        self.name = name
        self.n = n
        self.m = m
        self.cm = {}
        
        for color, tl in m.items():
            for t in tl:
                assert len(t) == self.n
                for e in t:
                    assert e < 3
                index = self.get_index(t)
                assert index not in self.cm
                self.cm[self.get_index(t)] = color
                
        tc = 3 ** self.n
        ec = sum(map(lambda a: len(a), self.m.values()))
        assert tc == ec
            
    def get_tuple(self, l):
        r = [0] * self.n
        for i in range(self.n - 1, -1, -1):
            r[i] = l % 3
            l //= 3
        return r
    
    def mate(self, index1, index2):        
        def mate_inner(t1, t2, c, p, i):
            if i >= self.n:
                return [(self.get_index(c), p)]
            n1, n2 = t1[i], t2[i]
            r = []
            if n1 == 0:
                if n2 == 0:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p, i + 1)
                elif n2 == 1:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                elif n2 == 2:
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p, i + 1)
            elif n1 == 1:
                if n2 == 1:
                    c[i] = 0
                    r += mate_inner(t1, t2, c, p / 4, i + 1)
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p / 4, i + 1)
                elif n2 == 2:
                    c[i] = 1
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p / 2, i + 1)
            elif n1 == 2:
                if n2 == 2:
                    c[i] = 2
                    r += mate_inner(t1, t2, c, p, i + 1)

            return r
        return mate_inner(self.get_tuple(index1), self.get_tuple(index2), [None] * self.n, 1.0, 0)
    
    def mate_definite(self, index1, index2):
        [i1, i2] = sorted([index1, index2])
        r = self.mate(i1, i2)
        m = {}
        results = []
        for t in r:
            i, p = t
            c = self.cm[i]
            if c not in m:
                m[c] = []
            m[c].append((i, p))
        for c, t in m.items():
            if len(t) == 1:
                results += t
        return results, (i1, i2)
    
    def get_color_str(self, i):
        return "{} ({})".format(self.cm[i], '-'.join(map(lambda a: str(a), self.get_tuple(i))))

    
    def breeding_guide(self, cross_slots, clone_slots, store_slots, target_count):
        pool = {}
        
        i_cross_slots = list(map(lambda a: None if a is None else (self.get_index(a[0]), self.get_index(a[1])), cross_slots))
        i_clone_slots = list(map(lambda a: None if a is None else self.get_index(a), clone_slots))
        i_store_slots = list(map(lambda a: (self.get_index(a[0]), a[1]), store_slots))
            
        def add_to_pool(cindex, count=1):
            if cindex not in pool:
                pool[cindex] = 0
            pool[cindex] += count
        
        for slot in i_cross_slots:
            if slot is not None:
                add_to_pool(slot[0])
                add_to_pool(slot[1])
                
        for slot in i_clone_slots:
            if slot is not None:
                add_to_pool(slot)

        for (cindex, count) in i_store_slots:
            add_to_pool(cindex, count)
        
        clones_needed = {}
        for k, v in pool.items():
            if v < target_count:
                clones[k] = target_count - v
                
        all_color_indicies = list(pool.keys())
        new_breeds = []
        for i in range(len(all_color_indicies)):
            for j in range(i + 1):
                c_results, sorted_parents = self.mate_definite(all_color_indicies[i], all_color_indicies[j])
                for c_result in c_results:
                    (c_index, pp) = c_result
                    if c_index not in pool:
                        new_breeds.append((pp, c_index, sorted_parents))
        new_breeds.sort(reverse=True)        
        new_definite_breeds = {} # breeds with 100%, so only one needed
        
        n_cross_slots = [(None, None)] * len(i_cross_slots)
        n_clone_slots = [(None, None)] * len(i_clone_slots)
        
        def try_take_from_pool(parents):
            if parents[0] == parents[1]:
                if pool[parents[0]] >= 2:
                    pool[parents[0]] -= 2
                    return True
            elif pool[parents[0]] >= 1 and pool[parents[1]] >= 1:
                pool[parents[0]] -= 1
                pool[parents[1]] -= 1
                return True
            return False
        
        def try_take_from_pool_single(p):
            if pool[p] >= 1:
                pool[p] -= 1
                return True
            return False
                        
        # cross slot
        for new_breed in new_breeds:
            (pp, c_index, parents) = new_breed
            if c_index in new_definite_breeds:
                continue
            
            # match existing cross_breeds
            materialized = False
            for i, cross_slot in enumerate(i_cross_slots):
                if parents == cross_slot:
                    if try_take_from_pool(parents):
                        n_cross_slots[i] = (parents, None)
                        materialized = True
                        break
            
            # find an empty slot
            if not materialized:
                for i, cross_slot in enumerate(i_cross_slots):
                    if cross_slot is not None:
                        continue
                    if try_take_from_pool(parents):
                        n_cross_slots[i] = (parents, ('+', parents))
                        materialized = True
                        break
            
            if materialized:
                if pp == 1.0:
                    new_definite_breeds[c_index] = True
                    
        # remove from old cross_slot
        for i, oslot in enumerate(i_cross_slots):
            if oslot is None:
                continue
            nslot = n_cross_slots[i]
            if nslot is None:
                n_cross_slots[i] = (None, ('-', oslot))
        
        # clone slot
        for k, v in clones_needed.items():
            # match existing clone_slots
            for i, clone_slot in enumerate(i_clone_slots):
                if v <= 0:
                    break
                if clone_slot == k:
                    if try_take_from_pool_single(k):
                        v -= 1
                        n_clone_slots[i] = (k, None)
                        continue
                        
            # match empty clone_slots
            for i, clone_slot in enumerate(i_clone_slots):
                if v <= 0:
                    break
                if clone_slot is None:
                    if try_take_from_pool_single(k):
                        v -= 1
                        n_clone_slots[i] = (k, ('+', k))
                        continue
        # remove from old clone slot
        for i, oslot in enumerate(i_clone_slots):
            if oslot is None:
                continue
            nslot = n_clone_slots[i]
            if nslot is None:
                n_clone_slots[i] = (None, ('-', oslot))
        
        n_store_slots = [None] * len(i_store_slots)
        for i, (cindex, count) in enumerate(i_store_slots):
            n_store_slots[i] = (cindex, pool[cindex])
            pool[cindex] = 0
        for cindex, c in pool.items():
            if c > 0:
                n_storage_slots.append((cindex, c))
        return n_cross_slots, n_clone_slots, n_store_slots
    
    def print_breeding_guide(self, cross_slots, clone_slots, storage_slots, target_count = 1):
        (n_cross_slots, n_clone_slots, n_store_slots) = self.breeding_guide(cross_slots, clone_slots, storage_slots, target_count)
        def get_pair_str(nsv):
            return 'E' if nsv is None else '{} x {}'.format(self.get_color_str(nsv[0]), self.get_color_str(nsv[1]))
        
        print(self.name)
        print("=====================")
        print("Cross breeding slots:")
        print("---------------------")
        for i, (nsv, delta) in enumerate(n_cross_slots):
            print("{}: {}".format(i+1, get_pair_str(nsv)))
            if delta is not None:
                (op, v) = delta
                print ("\t{} {}".format(op, get_pair_str(v)))
            if nsv is not None:
                for (c, p) in self.mate_definite(nsv[0], nsv[1])[0]:
                    print("\t{} [{}%]".format(self.get_color_str(c), p * 100))
        print("---------------------")                   
        print("Clone slots:")
        print("---------------------")
        for i, (nsv, delta) in enumerate(n_clone_slots):
            print("{}: {}".format(i+1, 'E' if nsv is None else self.get_color_str(nsv)))
            if delta is not None:
                (op, v) = delta
                print ("\t{} {}".format(op, self.get_color_str(v)))
        print( n_store_slots)
        return
        for gtype in l:
            cindex = self.get_index(gtype)
            print("{}: {}".format(pool[cindex], self.get_g_color_str(cindex)))
        print("-----")
        for entry in data:
            (cindex, (p1, p2), pp) = entry
            print("{}: {}: {}[{}] x {}[{}] [{}%]".format(self.get_g_color_str(cindex), self.name, self.get_g_color_str(p1), pool[p1], self.get_g_color_str(p2), pool[p2], pp * 100))
            
    

In [283]:
windflower = Flower('Windflower', 3,{
    "Red": [
        (1,0,0),(1,0,1),
        (2,0,0),(2,0,1),(2,1,0),(2,1,1)],
    "Blue": [
        (1,0,2),
        (0,0,2),(0,1,2)],
    "Pink": [
        (1,1,0),(1,1,1),(1,1,2),
        (2,2,0),(2,2,1)],
    "Orange": [
        (1,2,0),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Purple": [
        (2,0,2),(2,1,2),(2,2,2)],
    "White": [
        (0,0,0),(0,0,1)]
    })
windflower.print_breeding_guide(
    [
        None,
        ((1,0,1),(1,0,1)),
        ((0,0,2),(1,1,0)),
        None,
        None,
        None,
    ],
    [
        (0,1,1),
        None
    ],
    [
        ((2,0,0),9),
        ((0,2,0),3),
        ((0,0,1),3),
        ((1,1,0),1),
        ((0,0,2),0),
        ((1,0,1),0),
        ((0,1,1),1)
    ],
    2)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-283-cece299740ab>", line 19, in <module>
    windflower.print_breeding_guide(
  File "<ipython-input-282-9ce23b1972bd>", line 230, in print_breeding_guide
    (n_cross_slots, n_clone_slots, n_store_slots) = self.breeding_guide(cross_slots, clone_slots, storage_slots, target_count)
  File "<ipython-input-282-9ce23b1972bd>", line 120, in breeding_guide
    clones[k] = target_count - v
NameError: name 'clones' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError'

NameError: name 'clones' is not defined

In [284]:
pansy = Flower("Pansy", 3,{
    "Red": [
        (1,0,0),(1,0,1),
        (2,0,0),(2,0,1),(2,1,0),(2,1,1)],
    "Blue": [
        (1,0,2),
        (0,0,2),(0,1,2)],
    "Orange": [
        (1,1,0),(1,1,1),(1,1,2),
        (2,2,0),(2,2,1)],
    "Yellow": [
        (1,2,0),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Purple": [
        (2,0,2),(2,1,2),(2,2,2)],
    "White": [
        (0,0,0),(0,0,1)]
    })
pansy.print_breeding_guide(
    [
        None,
        None,
        None,
        None,
        None,
        None,
    ],
    [
        None
    ],
    [
        (1, (2,0,0),6),
        (2, (0,2,0),6),
        (3, (0,0,1),13),
    ],
    2)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-284-7a3329200d3e>", line 19, in <module>
    pansy.print_breeding_guide(
  File "<ipython-input-282-9ce23b1972bd>", line 230, in print_breeding_guide
    (n_cross_slots, n_clone_slots, n_store_slots) = self.breeding_guide(cross_slots, clone_slots, storage_slots, target_count)
  File "<ipython-input-282-9ce23b1972bd>", line 98, in breeding_guide
    i_store_slots = list(map(lambda a: (self.get_index(a[0]), a[1]), store_slots))
  File "<ipython-input-282-9ce23b1972bd>", line 98, in <lambda>
    i_store_slots = list(map(lambda a: (self.get_index(a[0]), a[1]), store_slots))
  File "<ipython-input-282-9ce23b1972bd>", line 6, in get_index
    return reduce(lambda a, b: a * 3 + b, l, 0)
TypeError: reduce() arg 2 must support iteration

Durin

TypeError: reduce() arg 2 must support iteration

In [5]:
cosmos = Flower(3,{
    "Pink": [
        (1,0,0),(1,0,1),(1,0,2),(1,1,2)],
    "Orange": [
        (1,1,0),(1,1,1),(1,2,0),(1,2,1),(1,2,2),
        (2,1,0),(2,1,1)],
    "Red": [
        (2,0,0),(2,0,1),(2,0,2),(2,1,2),(2,2,2)],
    "Black": [
        (2,2,0),(2,2,1)],
    "Yellow": [
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "White": [
        (0,0,0),(0,0,1),(0,0,2),(0,1,2)]
    }, [
        (2,0,0),
        (0,2,1),
        (0,0,1)
    ], [
        (1,1,2),
        (2,1,1)
    ])
cosmos.print_definites()

TypeError: __init__() missing 1 required positional argument: 'si'

In [11]:
tulip = Flower("Tulip", 3,{
    "Red": [
        (1,0,0),
        (2,0,1),(2,0,2),(2,1,1),(2,1,2)],
    "Pink": [
        (1,0,1)],
    "White": [
        (1,0,2),
        (0,0,0),(0,0,1),(0,0,2),(0,1,2)],
    "Orange": [
        (1,1,0),(1,2,0)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Black": [
        (2,0,0),(2,1,0)],
    "Purple": [
        (2,2,0),(2,2,1),(2,2,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,1),
        (1,2,0),
        (2,1,0)
    ])
tulip.print_breeding_guide([
    (0,0,1),
    (2,0,1),
    (0,2,0),
    (1,1,1),
    (1,0,1),
    (2,0,0),
    (1,0,0),
    (1,1,0)
])

1: White (0-0-1)
2: Red (2-0-1)
3: Yellow (0-2-0)
4: Yellow (1-1-1)
5: Pink (1-0-1)
6: Black (2-0-0)
7: Red (1-0-0)
8: Orange (1-1-0)
-----
Yellow (0-1-0): Tulip: Yellow (0-2-0)[3] x Red (1-0-0)[7] [50.0%]
White (1-0-2): Tulip: White (0-0-1)[1] x Red (2-0-1)[2] [25.0%]
White (0-0-0): Tulip: Red (1-0-0)[7] x Red (1-0-0)[7] [25.0%]
White (1-0-2): Tulip: Pink (1-0-1)[5] x Red (2-0-1)[2] [12.5%]
White (0-0-0): Tulip: Red (1-0-0)[7] x Orange (1-1-0)[8] [12.5%]
Yellow (0-1-0): Tulip: Red (1-0-0)[7] x Orange (1-1-0)[8] [12.5%]
White (1-0-2): Tulip: Yellow (1-1-1)[4] x Red (2-0-1)[2] [6.25%]
White (0-0-0): Tulip: Orange (1-1-0)[8] x Orange (1-1-0)[8] [6.25%]
Purple (2-2-0): Tulip: Orange (1-1-0)[8] x Orange (1-1-0)[8] [6.25%]


In [21]:
mum = Flower("Mum", 3,{
    "Pink": [
        (1,0,0),(1,0,1),(1,0,2),
        (1,1,2)],
    "Yellow": [
        (1,1,0),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Red": [
        (1,1,1),
        (2,0,0),(2,0,1),(2,0,2),(2,1,2),(2,2,2)],
    "Purple": [
        (1,2,0),(1,2,1),(1,2,2),
        (2,1,0),(2,1,1),
        (0,0,2)],
    "Green": [
        (2,2,0),(2,2,1)],
    "White": [
        (0,0,0),(0,0,1),(0,1,2)]
    }, [
        (2,0,0),
        (0,2,0),
        (0,0,1)
    ], [
        (1,1,2),
        (2,1,1)
    ])
mum.print_breeding_guide([
    (0,0,1),
    (2,0,0),
    (0,2,0),
    (0,0,2),
    (1,1,0),
    (0,1,1),
    (1,0,1),
    (1,2,0)
])

1: White (0-0-1)
2: Red (2-0-0)
3: Yellow (0-2-0)
4: Purple (0-0-2)
5: Yellow (1-1-0)
6: Yellow (0-1-1)
7: Pink (1-0-1)
8: Purple (1-2-0)
-----
Red (1-1-1): Mum: Purple (0-0-2)[4] x Purple (1-2-0)[8] [50.0%]
Purple (2-1-0): Mum: Purple (1-2-0)[8] x Red (2-0-0)[2] [50.0%]
Pink (1-0-0): Mum: Yellow (1-1-0)[5] x Red (2-0-0)[2] [25.0%]
Red (1-1-1): Mum: Purple (0-0-2)[4] x Yellow (1-1-0)[5] [25.0%]
Red (1-1-1): Mum: White (0-0-1)[1] x Purple (1-2-0)[8] [25.0%]
Red (1-1-1): Mum: Yellow (0-1-1)[6] x Red (2-0-0)[2] [25.0%]
Red (1-1-1): Mum: Yellow (0-2-0)[3] x Pink (1-0-1)[7] [25.0%]
Red (1-1-1): Mum: Pink (1-0-1)[7] x Purple (1-2-0)[8] [25.0%]
Purple (2-1-0): Mum: Yellow (1-1-0)[5] x Red (2-0-0)[2] [25.0%]
Green (2-2-0): Mum: Purple (1-2-0)[8] x Purple (1-2-0)[8] [25.0%]
Pink (1-0-0): Mum: Yellow (1-1-0)[5] x Yellow (1-1-0)[5] [12.5%]
Red (1-1-1): Mum: White (0-0-1)[1] x Yellow (1-1-0)[5] [12.5%]
Red (1-1-1): Mum: Yellow (0-1-1)[6] x Pink (1-0-1)[7] [12.5%]
Red (1-1-1): Mum: Yellow (0-1-1)[6

In [14]:
lily = Flower("Lily", 3,{
    "Pink": [
        (1,0,1),
        (2,0,2),(2,1,2)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,2,0),(0,2,1)],
    "Red": [
        (1,0,0),
        (2,0,1),(2,1,1)],
    "Orange": [
        (1,1,0),(1,2,0),
        (2,2,0),(2,2,1)],
    "Black": [
        (2,0,0),(2,1,0)],
    "White": [
        (0,0,0),(0,0,1),(0,0,2),(0,1,1),(0,1,2),(0,2,2),
        (1,0,2),
        (2,2,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,2)
    ], [
        (2,2,1),
        (2,1,0),
        (2,1,2)
    ])
lily.print_breeding_guide([
    (0,0,2),
    (2,0,1),
    (0,2,0),
    (0,1,1),
    (1,1,0),
    (1,0,1),
    (1,0,0),
    (2,0,2),
    (1,0,2)
])

1: White (0-0-2)
2: Red (2-0-1)
3: Yellow (0-2-0)
4: White (0-1-1)
5: Orange (1-1-0)
6: Pink (1-0-1)
7: Red (1-0-0)
8: Pink (2-0-2)
9: White (1-0-2)
-----
Yellow (1-1-1): Lily: Yellow (0-2-0)[3] x Pink (2-0-2)[8] [100.0%]
Yellow (1-1-1): Lily: Yellow (0-2-0)[3] x Red (2-0-1)[2] [50.0%]
Yellow (1-1-1): Lily: Yellow (0-2-0)[3] x White (1-0-2)[9] [50.0%]
White (0-0-1): Lily: White (0-0-2)[1] x Red (1-0-0)[7] [50.0%]
Yellow (0-1-0): Lily: Yellow (0-2-0)[3] x Red (1-0-0)[7] [50.0%]
Yellow (1-1-1): Lily: White (0-0-2)[1] x Orange (1-1-0)[5] [25.0%]
Yellow (1-1-1): Lily: White (1-0-2)[9] x Orange (1-1-0)[5] [25.0%]
Yellow (1-1-1): Lily: Orange (1-1-0)[5] x Pink (2-0-2)[8] [25.0%]
Black (2-0-0): Lily: Red (2-0-1)[2] x Red (2-0-1)[2] [25.0%]
Black (2-0-0): Lily: Red (1-0-0)[7] x Red (1-0-0)[7] [25.0%]
Black (2-0-0): Lily: Red (1-0-0)[7] x Red (2-0-1)[2] [25.0%]
White (0-0-0): Lily: Red (1-0-0)[7] x Red (1-0-0)[7] [25.0%]
White (0-0-1): Lily: Red (1-0-0)[7] x White (1-0-2)[9] [25.0%]
Yellow (1-1

In [13]:
hyacinth = Flower("Hyacinth", 3,{
    "Pink": [
        (1,0,1)],
    "Yellow": [
        (1,1,1),(1,1,2),(1,2,1),(1,2,2),
        (0,1,0),(0,1,1),(0,2,0),(0,2,1),(0,2,2)],
    "Red": [
        (1,0,0),
        (2,0,0),(2,0,1),(2,0,2),(2,1,1),(2,1,2)],
    "Orange": [
        (1,1,0),(1,2,0)],
    "Purple": [
        (2,2,0),(2,2,1),(2,2,2)],
    "Blue": [
        (0,0,2),
        (2,1,0)],
    "White": [
        (0,0,0),(0,0,1),(0,1,2),
        (1,0,2)]
    }, [
        (2,0,1),
        (0,2,0),
        (0,0,1)
    ], [
        (1,0,1),
        (2,1,0)
    ])
hyacinth.print_breeding_guide([
    (0,0,1),
    (2,0,1),
    (0,2,0),
    (1,1,1),
    (1,0,1),
    (1,0,0),
    (1,1,0),
    (0,0,2)
])

1: White (0-0-1)
2: Red (2-0-1)
3: Yellow (0-2-0)
4: Yellow (1-1-1)
5: Pink (1-0-1)
6: Red (1-0-0)
7: Orange (1-1-0)
8: Blue (0-0-2)
-----
Yellow (0-1-1): Hyacinth: Blue (0-0-2)[8] x Yellow (0-2-0)[3] [100.0%]
White (1-0-2): Hyacinth: Blue (0-0-2)[8] x Red (2-0-1)[2] [50.0%]
Yellow (0-1-0): Hyacinth: Yellow (0-2-0)[3] x Red (1-0-0)[6] [50.0%]
White (1-0-2): Hyacinth: White (0-0-1)[1] x Red (2-0-1)[2] [25.0%]
White (0-0-0): Hyacinth: Red (1-0-0)[6] x Red (1-0-0)[6] [25.0%]
White (1-0-2): Hyacinth: Pink (1-0-1)[5] x Red (2-0-1)[2] [12.5%]
White (0-0-0): Hyacinth: Red (1-0-0)[6] x Orange (1-1-0)[7] [12.5%]
Yellow (0-1-0): Hyacinth: Red (1-0-0)[6] x Orange (1-1-0)[7] [12.5%]
Blue (2-1-0): Hyacinth: Red (1-0-0)[6] x Orange (1-1-0)[7] [12.5%]
Blue (2-1-0): Hyacinth: Orange (1-1-0)[7] x Orange (1-1-0)[7] [12.5%]
Blue (2-1-0): Hyacinth: Orange (1-1-0)[7] x Red (2-0-1)[2] [12.5%]
White (1-0-2): Hyacinth: Yellow (1-1-1)[4] x Red (2-0-1)[2] [6.25%]
White (0-0-0): Hyacinth: Orange (1-1-0)[7] x Ora

In [250]:
rose = Flower('Rose', 4,{
    "Red": [
        (1,0,0,0),(1,0,1,0),(1,0,2,0),
        (1,1,1,0),(1,1,2,0),
        (1,2,2,0),
        (2,0,0,1),(2,0,1,1),(2,0,2,1),
        (2,1,1,0),(2,1,1,1),(2,1,2,1),
        (2,2,2,1)
    ],
    "Pink": [
        (1,0,0,1),(1,0,1,1),(1,0,2,1),
        (1,1,1,1),(1,1,2,1),
        (1,2,2,1),
        (2,0,0,2),(2,0,1,2),(2,0,2,2)
    ],
    "White": [
        (0,0,0,0),(0,0,0,1),(0,0,0,2),(0,0,1,0),(0,0,1,1),(0,0,1,2),
        (0,1,1,0),(0,1,1,1),(0,1,1,2),
        (0,2,2,0),(0,2,2,1),(0,2,2,2),
        (1,0,0,2),(1,0,1,2),
        (1,1,1,2),
        (1,2,2,2),
        (2,1,1,2),
        (2,2,2,2)
    ],
    "Purple":[
        (0,0,2,0),(0,0,2,1),(0,0,2,2),
        (0,1,2,0),(0,1,2,1),(0,1,2,2),
        (1,0,2,2),
        (1,1,2,2),
        (2,1,2,2)
    ],
    "Orange": [
        (1,1,0,0),
        (1,2,0,0),(1,2,1,0),
        (2,1,0,0),(2,1,0,1),
        (2,2,0,0),(2,2,0,1),(2,2,1,0),(2,2,1,1)
    ],
    "Yellow": [
        (0,1,0,0),(0,1,0,1),(0,1,0,2),
        (0,2,0,0),(0,2,0,1),(0,2,0,2),(0,2,1,0),(0,2,1,1),(0,2,1,2),
        (1,1,0,1),(1,1,0,2),
        (1,2,0,1),(1,2,0,2),(1,2,1,1),(1,2,1,2),
        (2,1,0,2),
        (2,2,0,2),(2,2,1,2)
    ],
    "Black": [
        (2,0,0,0),(2,0,1,0),(2,0,2,0),
        (2,1,2,0)
    ],
    "Blue": [
        (2,2,2,0)
    ]})
rose.print_breeding_guide(
    [
        ((0,1,0,0),(1,0,0,0)),
        None,
        None,
        ((1,1,0,1),(2,0,0,1)),
        None,
        ((0,0,2,0),(2,0,0,0)),
        ((1,0,0,1),(2,0,0,1)),
        ((1,1,0,0),(2,0,0,2)),
        ((0,2,0,0),(1,0,1,1)),
        None,
    ],
    [
        None
    ],
    [
        ((2,0,0,1),2),
        ((0,2,0,0),10),
        ((0,0,1,0),5),
        ((0,1,1,0),2),
        ((1,1,0,0),1),
        ((1,1,0,1),1),
        ((0,1,0,0),0),
        ((0,0,2,0),0),
        ((2,0,0,0),0),
        ((2,0,0,2),0),
        ((1,0,0,1),0),
        ((1,0,1,1),0),
        ((1,0,0,0),0)
    ],
    2)

Rose
Cross breeding slots:
---------------------
1: Yellow (0-1-0-0) x Red (1-0-0-0)
	White (0-0-0-0) [25.0%]
	Yellow (0-1-0-0) [25.0%]
	Red (1-0-0-0) [25.0%]
	Orange (1-1-0-0) [25.0%]
2: E
3: E
4: Yellow (1-1-0-1) x Red (2-0-0-1)
	White (1-0-0-2) [6.25%]
	Black (2-0-0-0) [6.25%]
5: E
6: Purple (0-0-2-0) x Black (2-0-0-0)
	Red (1-0-1-0) [100.0%]
7: Pink (1-0-0-1) x Red (2-0-0-1)
	White (1-0-0-2) [12.5%]
	Black (2-0-0-0) [12.5%]
8: Orange (1-1-0-0) x Pink (2-0-0-2)
	Pink (1-0-0-1) [25.0%]
	Yellow (1-1-0-1) [25.0%]
	Red (2-0-0-1) [25.0%]
	Orange (2-1-0-1) [25.0%]
9: Yellow (0-2-0-0) x Pink (1-0-1-1)
	Orange (1-1-0-0) [12.5%]
	Red (1-1-1-0) [12.5%]
	Pink (1-1-1-1) [12.5%]
10: E
---------------------
Clone slots:
---------------------
1: E
---------------------
Storage slots:
---------------------
1: Red (2-0-0-1) x 2
2: Yellow (0-2-0-0) x 10
3: White (0-0-1-0) x 5
4: White (0-1-1-0) x 2
5: Orange (1-1-0-0) x 1
6: Yellow (1-1-0-1) x 1
7: Yellow (0-1-0-0) x 0
8: Purple (0-0-2-0) x 0
9: Blac